In [1]:
from LiveAuctionProcessor import LiveAuctionProcessor

In [11]:
auction_dict = {"auction_id":'210202139'}
lap = LiveAuctionProcessor(auction_dict)


In [12]:
lap.update()

5e554297b38ba1a5c05c4c9f
[{'bid': 59, 'usename': 'thelablady', 'is_bidomatic': True}]
5e5542a1b38ba1a5c05c4ca0
[{'bid': 59, 'usename': 'thelablady', 'is_bidomatic': True}, {'bid': 60, 'usename': 'BIDTOBUYCSM', 'is_bidomatic': False}]
5e5542a4b38ba1a5c05c4ca1
[{'bid': 59, 'usename': 'thelablady', 'is_bidomatic': True}, {'bid': 60, 'usename': 'BIDTOBUYCSM', 'is_bidomatic': False}, {'bid': 61, 'usename': 'thelablady', 'is_bidomatic': True}]
5e5542aeb38ba1a5c05c4ca2
[{'bid': 59, 'usename': 'thelablady', 'is_bidomatic': True}, {'bid': 60, 'usename': 'BIDTOBUYCSM', 'is_bidomatic': False}, {'bid': 61, 'usename': 'thelablady', 'is_bidomatic': True}, {'bid': 62, 'usename': 'BIDTOBUYCSM', 'is_bidomatic': False}]
5e5542b0b38ba1a5c05c4ca3
[{'bid': 59, 'usename': 'thelablady', 'is_bidomatic': True}, {'bid': 60, 'usename': 'BIDTOBUYCSM', 'is_bidomatic': False}, {'bid': 61, 'usename': 'thelablady', 'is_bidomatic': True}, {'bid': 62, 'usename': 'BIDTOBUYCSM', 'is_bidomatic': False}, {'bid': 63, 'usena